## Data Integration

This notebook includes steps for integrating of two tables 'tracks_sample.csv' and 'songs_sample.csv' based on the matching pairs. These two table has two different schemas. Thus, schema of the final table E is the union of these two table's schema.

### Loading libraries and reading data

In [1]:
import pandas as pd
import os
import re

fileDir = os.path.join('..','Stage 3','data')

songs = pd.read_csv(os.path.join(fileDir,'songs_sample.csv'))
tracks = pd.read_csv(os.path.join(fileDir,'tracks_sample.csv'))
matchIDPairs = pd.read_csv('final_predictions.csv')

# filtering the matched tuples from both dataset 
matchedTracks = tracks[tracks['id'].isin(list(matchIDPairs['ltable_id']))]
matchedSongs = songs[songs['id'].isin(list(matchIDPairs['rtable_id']))]

assert(len(matchedTracks)==len(matchedSongs))

In [2]:
matchedTracks.head()

,id,movie_title,year,episode,song_title,artists
5,262158,the porter wagoner show,1961.0,the osborne brothers (#1.517),the carroll county accident,porter wagoner
70,393455,claudia leitte: ao vivo em copacabana,2008.0,NaN,pensando em você,henrique cerqueira+claudia leitte
91,459080,greta,2009.0,NaN,i wanna die,jolie holland
217,426815,el crimen del padre amaro,2002.0,NaN,te odio,rudy pérez+joel numa+pablo montero
276,328752,22nd annual trumpet awards,2014.0,NaN,i need you now,smokie norful


In [3]:
matchedSongs.head()

,id,song_title,artists,year
147,509218,he can only hold her,amy winehouse,2006
154,218585,last train home,pat metheny group,1987
179,261294,soverato,minus 8,2004
351,958721,sweet talkin' woman,electric light orchestra,1977
476,679231,god don't never change,blind willie johnson,1989


In [4]:
matchIDPairs.head()

,Unnamed: 0,id,ltable_id,rtable_id
0,309,906585,253443,260085
1,196,591561,723561,68786
2,246,740185,338596,635283
3,261,788096,713603,150365
4,37,114823,246156,315410


### Merging two tables 

In [10]:
#a helper function to extract substring
def extractString(st):
    text = str(st).split('\n')[0]
    m = re.search('\D+',text)
    if m:
        return m.group(0)

#Schema of the merged table
E = pd.DataFrame(columns = ['movie_title','year','episode','song_title','artists'])

for index, row in matchIDPairs.iterrows():
    left_entry = matchedTracks[matchedTracks['id']==row['ltable_id']]
    right_entry = matchedSongs[matchedSongs['id']==row['rtable_id']]
    
    assert(len(left_entry)==1)
    assert(len(right_entry)==1)
    
    if(math.isnan(left_entry['year'])):
            left=0
    else:
            left = int(float(str(left_entry['year']).split('\n')[0].split(' ')[4]))
    if(math.isnan(right_entry['year'])):
            right=0
    else:
            right = int(float(str(right_entry['year']).split('\n')[0].split(' ')[4]))
    
    if(left > right):
            year =  left
    elif (left < right):
            year =  right
    else:
        year =  left
    
    #for song title, larger length value is chosen if two value doesn't have exact string match
    left = extractString(left_entry['song_title'])
    right = extractString(right_entry['song_title'])
    
    if(len(left)!=len(right)):
        if(len(left) > len(right)):
            song_title = left
        else:
            song_title = right
    else:
        song_title = left
    
    #for artist, larger length value is chosen if two value doesn't have exact string match
    left = extractString(left_entry['artists'])
    right = extractString(right_entry['artists'])
    
    if(len(left)!=len(right)):
        if(len(left) > len(right)):
            artists = left
        else:
            artists = right
    else:
        artists = left
    
    #since movie and episode are unique attributes in the left table, keeping the value as it is
    movie_title = extractString(left_entry['movie_title'])
    episode = extractString(left_entry['episode'])
    
    #creating an entry for table E with all values
    entry = pd.Series([movie_title, year, episode, song_title, artists], index=['movie_title','year','episode','song_title','artists'])
    
    #appending the merged value to table E
    E = E.append(entry, ignore_index=True)

In [11]:
E.head()

,movie_title,year,episode,song_title,artists
0,the marty stuart show,2008.0,emmy lou harris (#,hearts like ours (album version),connie smith
1,wholl stop the rain,1978.0,NaN,gimme some lovin' (re-recorded),steve winwood+muff winwood+spencer davis+t...
2,about last night...,1990.0,NaN,words into action,mike leeson+peter vale+jermaine jackson
3,vidas en vivo,2008.0,NaN,niños,pedro guerra
4,the go!! show,1964.0,(#,i (who have nothing),jerry leiber+mike stoller+carlo donida+mog...


In [14]:
#Writing the table E to file
E.to_csv('Merged table E.csv',sep=',',index=False)